# Data viz

In [ ]:
import folium
import pandas as pd
import numpy as np
from geopy import GeoNames

In [ ]:
grants = pd.read_csv('P3_GrantExport.csv', sep=";")
grants['University'] = grants['University'].astype('str').apply(lambda s: s.split(' -')[0])

## TODO: replace institution name with city
@ Alexis 

In [ ]:
grants = grants.replace('', np.nan)
grants = grants.replace('nan', np.nan)
grants

In [ ]:
grants[pd.isnull(grants.Institution) & pd.isnull(grants.University)].shape

We can't do much about the rows without a University or an Institution field. We won't consider them.

In [ ]:
grants = grants[~(pd.isnull(grants.Institution) & pd.isnull(grants.University))]

We create a dictionnaire which stores the canton of each university/institution. We get the results from the GeoNames Webservice using `geopy`.

In [ ]:
unis = grants['University'].unique()
# we get the institution if there is no field in University
institutions = grants[pd.isnull(grants.University) & (~pd.isnull(grants.Institution))]['Institution'].unique()
unis = unis[~pd.isnull(unis)]
institutions = institutions[~pd.isnull(institutions)]

In [ ]:
unis

In [ ]:
geolocator = GeoNames(country_bias='CH', username='ada_dream_team', timeout=1)

In [ ]:
cantonDic = {}

for uni in unis:
    location = geolocator.geocode(uni)
    if location:
        location = location.address.split(', ')
        if len(location) > 2:
            address = location[1]
        else:
            address = np.nan
    else:
        address = np.nan
    cantonDic[uni] = address
'''
for instit in institutions:
    location = geolocator.geocode(instit)
    if location:
        location = location.address.split(', ')
        if len(location) > 2:
            address = location[1]
        else:
            address = np.nan
    else:
        address = np.nan
    cantonDic[instit] = address  
'''

In [ ]:
location = geolocator.geocode('HEG')

In [ ]:
location

In [ ]:
def get_canton(df):
    if (not pd.isnull(df['University'])):
        address = cantonDic[df['University']]
    else:
        address = cantonDic[df['Institution']]
    if address:
        return address
    else:
        return np.nan

In [ ]:
grants['Canton'] = grants.apply(get_canton, axis=1)